In [1]:
import sys; sys.path.append('..')
import random, math, os
import pyzx as zx
from fractions import Fraction
from functools import reduce
import numpy as np
%config InlineBackend.figure_format = 'svg'
zx.quantomatic.quantomatic_location = r'C:\Users\John\Desktop\Quantomatic.jar'
zx.tikz.tikzit_location = r'C:\Users\John\Documents\tikzit\tikzit.exe'

In [174]:
def make_stabilizer_tableau(c):
    q = c.qubits
    m = zx.Mat2.id(2*q)
    def add_PHASE(i):
        m.col_add(i,q+i)
    def add_HAD(i):
        m.col_swap(i,q+i)
    def add_CNOT(i,j):
        m.col_add(i,j)
        m.col_add(q+j,q+i)
    for g in c.gates:
        if isinstance(g, zx.gates.ZPhase) and g.phase not in (0,1):
            add_PHASE(g.target)
        if isinstance(g, zx.gates.XPhase) and g.phase not in (0,1):
            add_HAD(g.target)
            add_PHASE(g.target)
            add_HAD(g.target)
        if g.name == "HAD":
            add_HAD(g.target)
        if g.name == "CNOT":
            add_CNOT(g.control,g.target)
        if g.name == "CZ":
            add_HAD(g.target)
            add_CNOT(g.control,g.target)
            add_HAD(g.target)
    return m

In [3]:
def generate_clifford_circuit(qubits, depth, p_cnot=0.3, p_t=0):
    p_s = 0.5*(1.0-p_cnot-p_t)
    p_had = 0.5*(1.0-p_cnot-p_t)
    c = zx.Circuit(qubits)
    for _ in range(depth):
        r = random.random()
        if r > 1-p_had:
            c.add_gate("HAD",random.randrange(qubits))
        elif r > 1-p_had-p_s:
            c.add_gate("S",random.randrange(qubits))
        elif r > 1-p_had-p_s-p_t:
            c.add_gate("T",random.randrange(qubits))
        else:
            tgt = random.randrange(qubits)
            while True:
                ctrl = random.randrange(qubits)
                if ctrl!=tgt: break
            c.add_gate("CNOT",tgt,ctrl)
    return c

In [15]:
c = generate_clifford_circuit(3,8)
zx.d3.draw(c)
print(make_stabilizer_tableau(c))

[ 1  0  0  1  0  1 ]
[ 0  1  0  0  0  0 ]
[ 1  0  1  0  0  0 ]
[ 0  0  0  1  0  1 ]
[ 0  0  0  0  1  0 ]
[ 0  0  0  0  0  1 ]


In [4]:
def make_hash(c):
    return hash(tuple(sum(make_stabilizer_tableau(c).data,[])))

def find_hit(c, checks):
    for labelling in [(0,1,2),(0,2,1),(1,2,0),(1,0,2),(2,0,1),(2,1,0)]:
        cp = c.copy()
        for g in cp.gates:
            g.target = labelling[g.target]
            if g.name == 'CNOT': g.control = labelling[g.control]
        h = make_hash(cp)
        if any(h in l for l in checks): return True
    return False

In [90]:
# Generate layers by minimal number of Clifford gates needed
q = 3
id_circuit = zx.Circuit(q)
layers = [{make_hash(id_circuit):id_circuit}]
previous = layers[0]
previous2 = layers[0]

In [143]:
while len(layers)<12:
    newlayer = dict()
    for c in previous.values():
        for g in ["S","HAD"]:
            for i in range(q):
                cp = c.copy()
                cp.add_gate(g,i)
                if not find_hit(cp,(previous,previous2,newlayer)):
                    h = make_hash(cp)
                    newlayer[h] = cp
                    if len(newlayer) % 10000 == 0: print(len(newlayer))
        for i in range(q):
            for j in range(q):
                if i == j: continue
                cp = c.copy()
                cp.add_gate("CNOT",i,j)
                if not find_hit(cp,(previous,previous2,newlayer)):
                    h = make_hash(cp)
                    newlayer[h] = cp
                    if len(newlayer) % 10000 == 0: print(len(newlayer))
    print("There are", len(newlayer), "circuits of size", len(layers))
    previous2 = previous
    previous = newlayer
    layers.append(previous)

There are 4243 circuits of size 11


In [225]:
def find_representative(c, layers):
    for labelling in [(0,1,2),(0,2,1),(1,2,0),(1,0,2),(2,0,1),(2,1,0)]:
        cp = c.copy()
        for g in cp.gates:
            g.target = labelling[g.target]
            if g.name == 'CNOT': g.control = labelling[g.control]
        h = make_hash(cp)
        for l in layers:
            if h in l: 
                inverse_labelling = tuple(labelling.index(i) for i in range(3))
                cp = l[h].copy()
                for g in cp.gates:
                    g.target = inverse_labelling[g.target]
                    if g.name == 'CNOT': g.control = inverse_labelling[g.control]
                return cp
    raise ValueError("No match found")

def correct_representative(c1,c2):
    c = c1.copy()
    c.add_circuit(c2.adjoint())
    g = c.to_graph()
    zx.clifford_simp(g,quiet=True)
    g.normalise()
    zx.id_simp(g,quiet=True)
    zx.to_rg(g)
    try:
        paulis = zx.Circuit.from_graph(g)
    except:
        print(zx.compare_tensors(g,c,False))
        zx.d3.draw(g)
        raise
    paulis.add_circuit(c2)
    return paulis

# random.seed(1370)
# c = generate_clifford_circuit(3,70)
# zx.d3.draw(c)
# c_opt = find_representative(c, layers)
# #zx.d3.draw(c_opt)
# c2 = correct_representative(c,c_opt)
# zx.d3.draw(c2)
# print(zx.compare_tensors(c,c2,False))

In [27]:
# Layers by 2-qubit gates
q = 3
id_circuit = zx.Circuit(q)
layers = [{make_hash(id_circuit):id_circuit}]
current = layers[0]
to_check = current

while True:
    new_ones = dict()
    for c in to_check.values():
        for g in ("S","HAD"):
            for i in range(q):
                cp = c.copy()
                cp.add_gate(g,i)
                if not find_hit(cp, layers+[new_ones]):
                    h = make_hash(cp)
                    new_ones[h] = cp
    if new_ones: 
        current.update(new_ones)
        to_check = new_ones
    else: 
        print("There are", len(current), "circuits with 0 CNOT gates")
        break

There are 56 circuits with 0 CNOT gates


In [30]:
while len(layers)<8:
    new_layer = dict()
    marker = 5000
    layers.append(new_layer)
    for c in current.values():
        for i in range(q):
            for j in range(q):
                if i == j: continue
                cp = c.copy()
                cp.add_gate("CNOT",i,j)
                if not find_hit(cp, layers):
                    h = make_hash(cp)
                    new_layer[h] = cp
                    new_circs = {h: cp}
                    while True:
                        cs = dict()
                        for d in new_circs.values():
                            for g in ("S","HAD"):
                                for k in (i,j):
                                    dp = d.copy()
                                    dp.add_gate(g,k)
                                    if not find_hit(dp, layers):
                                        h = make_hash(dp)
                                        cs[h] = dp
                                        new_layer[h] = dp
                        if cs:
                            new_circs.update(cs)
                        else: break
                    if len(new_layer) > marker: 
                        print("new_layer:", len(new_layer))
                        marker += 5000
    print("There are", len(new_layer), "circuits with", len(layers)-1, "CNOT gates")
    current = new_layer
    to_check = current

There are 76 circuits with 6 CNOT gates
There are 0 circuits with 7 CNOT gates


In [43]:
print([len(l) for l in layers])

[56, 1026, 15714, 101100, 110692, 15553, 76, 0]


In [263]:
qasm = "\n----------\n".join(c.to_qasm() for c in layers[2].values())
len(qasm)

2255262

In [293]:
def find_subcircuit(c, i, qubits):
    # i is pivot gate index
    # qubits should be a list of allowed qubit indices in the subcircuit
    gates = []
    before = c.gates[:i]
    after = []
    blocked = {}
    barrier = dict()
    for g in c.gates[i:]:
        if not g.name in ("CNOT", "CZ"):
            if g.target in barrier and barrier[g.target] == 'A': after.append(g)
            elif g.target in qubits:
                if g.target not in barrier:
                    if isinstance(g, zx.gates.ZPhase):
                        if g.target not in blocked:
                            blocked[g.target] = 'Z'
                        elif g.target in blocked and blocked[g.target] == 'X':
                            blocked[g.target] = 'A'
                        if g.phase.denominator <= 2:
                            gates.append(g)
                        else: # Not Clifford
                            barrier[g.target] = 'Z'
                            after.append(g)
                    elif g.name == 'HAD':
                        blocked[g.target] = 'A'
                        gates.append(g)
                    elif isinstance(g, zx.gates.XPhase):
                        if g.target not in blocked:
                            blocked[g.target] = 'X'
                        elif g.target in blocked and blocked[g.target] == 'Z':
                            blocked[g.target] = 'A'
                        gates.append(g)
                    else: raise Exception("unknown gate " + str(g))
                else:
                    if isinstance(g, zx.gates.ZPhase):
                        if barrier[g.target] == 'Z': 
                            if g.phase.denominator <= 2: gates.append(g)
                            else: after.append(g)
                        else: 
                            barrier[g.target] = 'A'
                            after.append(g)
                    elif g.name == 'HAD':
                        barrier[g.target] = 'A'
                        after.append(g)
                    elif isinstance(g, zx.gates.XPhase):
                        if barrier[g.target] == 'X': gates.append(g)
                        else: 
                            barrier[g.target] = 'A'
                            after.append(g)
            else:
                if g.target not in barrier: before.append(g)
                else:
                    if isinstance(g, zx.gates.ZPhase):
                        if barrier[g.target] == 'Z': before.append(g)
                        else:
                            barrier[g.target] == 'A'
                            after.append(g)
                    else:
                        barrier[g.target] = 'A'
                        after.append(g)
        elif g.name == "CNOT":
            if ((g.target not in barrier or barrier[g.target] == 'X') and 
                    (g.control not in barrier or barrier[g.control] == 'Z')):
                if g.target in qubits and g.control in qubits: 
                    gates.append(g)
                    if g.target not in blocked: blocked[g.target] = 'X'
                    elif blocked[g.target] == 'Z': blocked[g.target] = 'A'
                    if g.control not in blocked: blocked[g.control] = 'Z'
                    elif blocked[g.control] == 'X': blocked[g.control] = 'A'
                elif g.target not in qubits and g.control not in qubits:
                    before.append(g)
                elif g.target in qubits:
                    if g.target not in blocked: before.append(g)
                    elif blocked[g.target] == 'X': before.append(g)
                    else: 
                        barrier[g.target] = 'X'
                        barrier[g.control] = 'Z'
                        after.append(g)
                elif g.control in qubits:
                    if g.control not in blocked: before.append(g)
                    elif blocked[g.control] == 'Z': before.append(g)
                    else:
                        barrier[g.target] = 'X'
                        barrier[g.control] = 'Z'
                        after.append(g)
            else:
                after.append(g)
                if g.target not in barrier: barrier[g.target] = 'X'
                elif barrier[g.target] == 'Z':barrier[g.target] = 'A'
                if g.control not in barrier: barrier[g.control] = 'Z'
                elif barrier[g.control] == 'X': barrier[g.control] = 'A'
        elif g.name == "CZ": 
            if ((g.target not in barrier or barrier[g.target] == 'Z') and 
                    (g.control not in barrier or barrier[g.control] == 'Z')):
                if g.target in qubits and g.control in qubits: 
                    gates.append(g)
                    if g.target not in blocked: blocked[g.target] = 'Z'
                    elif blocked[g.target] == 'X': blocked[g.target] = 'A'
                    if g.control not in blocked: blocked[g.control] = 'Z'
                    elif blocked[g.control] == 'X': blocked[g.control] = 'A'
                elif g.target not in qubits and g.control not in qubits:
                    before.append(g)
                elif g.target in qubits:
                    if g.target not in blocked: before.append(g)
                    elif blocked[g.target] == 'Z': before.append(g)
                    else: 
                        barrier[g.target] = 'Z'
                        barrier[g.control] = 'Z'
                        after.append(g)
                elif g.control in qubits:
                    if g.control not in blocked: before.append(g)
                    elif blocked[g.control] == 'Z': before.append(g)
                    else:
                        barrier[g.target] = 'Z'
                        barrier[g.control] = 'Z'
                        after.append(g)
            else:
                after.append(g)
                if g.target not in barrier: barrier[g.target] = 'Z'
                elif barrier[g.target] == 'X':barrier[g.target] = 'A'
                if g.control not in barrier: barrier[g.control] = 'Z'
                elif barrier[g.control] == 'X': barrier[g.control] = 'A'
        else: raise Exception("Shouldn't get here")
        
    return before, gates, after

In [289]:
def convert_CZs(c):
    c2 = zx.Circuit(c.qubits)
    for g in c.gates:
        if g.name != "CZ":
            c2.add_gate(g)
        else:
            c2.add_gate("HAD",g.target)
            c2.add_gate("CNOT",g.control, g.target)
            c2.add_gate("HAD",g.target)
    return c2

def replace_circuit(c):
    count = c.twoqubitcount()
    if count == 0: return None
    for labelling in [(0,1,2),(0,2,1),(1,2,0),(1,0,2),(2,0,1),(2,1,0)]:
        cp = c.copy()
        for g in cp.gates:
            g.target = labelling[g.target]
            if g.name in ("CNOT", "CZ"): g.control = labelling[g.control]
        h = make_hash(cp)
        for l in layers[:count]:
            if h in l: 
                inverse_labelling = tuple(labelling.index(i) for i in range(3))
                cp = l[h].copy()
                for g in cp.gates:
                    g.target = inverse_labelling[g.target]
                    if g.name in ("CNOT", "CZ"): g.control = inverse_labelling[g.control]
                return correct_representative(c,cp)
    return None

def optimize_circuit(c):
    #c = zx.optimize.basic_optimization(c,False).to_basic_gates()
    #c = convert_CZs(c)
    c = c.copy()
    q = c.qubits
    rounds_to_go = 2
    while True:
        for q_1 in range(q):
            for q_2 in range(q_1+1,q):
                for q_3 in range(q_2+1,q):
                    qubits = (q_1,q_2,q_3)
                    index = 0
                    while True:
                        before, middle, after = find_subcircuit(c, index, qubits)
                        c2 = zx.Circuit(q)
                        c2.gates = before + middle + after
#                         if not zx.compare_tensors(c,c2,False):
#                             print(qubits, index)
#                             raise Exception("Not equal")
                        small = zx.Circuit(3)
                        for g in middle:
                            g2 = g.copy()
                            g2.target = qubits.index(g.target)
                            if g2.name in ("CNOT", "CZ"): g2.control = qubits.index(g.control)
                            small.add_gate(g2)
                        opt = replace_circuit(small)
                        if opt:
                            #print(middle)
                            #print(small.gates)
                            #print(opt.gates)
                            rounds_to_go = 2
                            #print("Found improvement on", qubits, "subcircuit")
                            #print(small.twoqubitcount(), "to", opt.twoqubitcount(), "CNOT gates")
                            for g in opt.gates:
                                g.target = qubits[g.target]
                                if g.name in ("CNOT", "CZ"): g.control = qubits[g.control]
                            c.gates = before + opt.gates + after
                        for i in range(len(after)):
                            if after[i].target in qubits:
                                if after[i].name in ("CNOT", "CZ"):
                                    if after[i].control in qubits:
                                        break
                                if isinstance(after[i], zx.gates.ZPhase) and after[i].phase.denominator > 2: continue
                                else: break
                        else: break
                        index = len(before) + len(middle) + i
        c = zx.optimize.basic_optimization(c,False).to_basic_gates()
        rounds_to_go -= 1
        if not rounds_to_go: break
    return c

In [296]:
c = zx.Circuit.load(r'..\circuits\fast\mod5_4_before')
zx.d3.draw(c)
g = c.to_graph()
zx.full_reduce(g)
c2 = zx.extract.modified_extract(g)
zx.d3.draw(c2)
c3 = zx.optimize.basic_optimization(c2.to_basic_gates()).to_basic_gates()
zx.d3.draw(c3)
c4 = optimize_circuit(c3)
zx.d3.draw(c4)
g = c.to_graph()
g = zx.teleport_reduce(g)
c5 = zx.Circuit.from_graph(g)
c5 = optimize_circuit(c5)
zx.d3.draw(c5)
g = c.to_graph()
g = zx.teleport_reduce(g)
c6 = zx.Circuit.from_graph(g)
c6 = zx.optimize.full_optimize(c6.to_basic_gates()).to_basic_gates()
zx.d3.draw(c6)

In [291]:
zx.compare_tensors(c3,c4,False)

True

In [245]:
seed = 1340
random.seed(seed)
c = generate_clifford_circuit(5,50,p_cnot=0.30)
print(c.stats())
zx.d3.draw(c)

Circuit  on 5 qubits with 50 gates.
        0 is the T-count
        50 Cliffords among which 
        14 2-qubit gates and 20 Hadamard gates.


In [250]:
c2 = optimize_circuit(c)
# before, middle, after = find_subcircuit(c, 10, (0,1,2))
# c2 = zx.Circuit(4)
# c2.gates = before + middle + after
#print(zx.compare_tensors(c,c2,False))
g = c2.to_graph(True)
print(c2.stats())
zx.d3.draw(g)

Circuit  on 5 qubits with 37 gates.
        0 is the T-count
        37 Cliffords among which 
        9 2-qubit gates and 11 Hadamard gates.


In [251]:
c_opt = zx.optimize.basic_optimization(c2,False).to_basic_gates()
print(c_opt.stats())
zx.d3.draw(c_opt)

Circuit  on 5 qubits with 29 gates.
        0 is the T-count
        29 Cliffords among which 
        9 2-qubit gates and 8 Hadamard gates.


In [252]:
c3 = optimize_circuit(c_opt)
print(c3.stats())
zx.d3.draw(c3)

Circuit  on 5 qubits with 29 gates.
        0 is the T-count
        29 Cliffords among which 
        9 2-qubit gates and 8 Hadamard gates.


In [217]:
g = c.to_graph()
zx.clifford_simp(g,quiet=True)
c3 = zx.optimize.basic_optimization(zx.extract.streaming_extract(g).to_basic_gates(),False)
print(c3.stats())

Circuit  on 10 qubits with 33 gates.
        0 is the T-count
        33 Cliffords among which 
        15 2-qubit gates and 8 Hadamard gates.


In [258]:
#Benchmark

qubits = 6
depth = 60
reps = 30

counts = []

for _ in range(reps):
    c = generate_clifford_circuit(7,40,p_cnot=0.3)
    c2 = optimize_circuit(c)
    c3 = zx.optimize.basic_optimization(c2,False).to_basic_gates()
    c4 = optimize_circuit(convert_CZs(c3))
    counts.append((c.twoqubitcount(),c2.twoqubitcount(),c3.twoqubitcount(),c4.twoqubitcount()))
    if c2.twoqubitcount() > c3.twoqubitcount():
        zx.d3.draw(c2)
        zx.d3.draw(c3)
a,b,c,d = 0,0,0,0
for i,j,k,l in counts:
    a += i
    b += j
    c += k
    d += l
print(a/reps, b/reps, c/reps, d/reps)

12.2 10.333333333333334 10.333333333333334 10.3
